## CUDA Matrix Multiplication

This project is used to perform the matrix multipliacation on CUDA using SGEMM algorithm, and comparing with the build-in CUBLAS implement. Different versions of code applying different levels of optimization techniques and show the difference in performance.

First set up the experiments parameters.

In [4]:
import numpy as np
import struct
from numpy.random import *

# parameters set up
N = 128
H = 128
W = 128
K = 128

Generate the data of two different Matrixes and save to the files for later usage.  
Note: the data should be in binary format and the data type is 32 bits float.

In [ ]:
# generate random data and write to file
Matrix1 = (np.array( rand(N * H * K ) )*2-1.0).astype(np.float32)
# Matrix1 = np.ones(N * H * K).astype(np.float32)
des = open("./data/Matrix1.bin","wb")
cnt = des.write(Matrix1)
des.close()

Matrix2 = (np.array( rand(N * K * W ) )*2-1.0).astype(np.float32)
# Matrix2 = (np.ones(N * K * W )).astype(np.float32)
des = open("./data/Matrix2.bin","wb")
cnt = des.write(Matrix2)
des.close()

Read in the data from the file.

In [5]:
#readin the data from the file
src = open("./data/Matrix1.bin","rb")
context = src.read()
# print(len(context))
real_context = struct.unpack(str(N * H * K)+'f',context)
M1 = np.array(real_context).reshape((N,H,K))


src = open("./data/Matrix2.bin","rb")
context = src.read()
# print(len(context))
real_context = struct.unpack(str(N * K * W)+'f',context)
M2 = np.array(real_context).reshape((N,K,W))

Excute the Matrix Multiplication using python program and get result.

In [8]:
def MatrixMulBatched(a,b):
    # a and b represents the matrix1 and matrix2
    N = a.shape[0]
    H = a.shape[1]
    K = a.shape[2]
    W = b.shape[2]
    c = np.zeros((N,H,W))
    for n in range(N):
        c[n] = a[n] @ b[n]
    return c

# input = np.ones(parameters1).astype(np.float32).reshape((Aside,Aside,128))
# kernel = np.ones(parameters2).astype(np.float32).reshape((128,128,3,3)).transpose((0,2,3,1))
Output = MatrixMulBatched(M1,M2)
print(Output.shape)

(128, 128, 128)
